In [1]:
import pandas as pd
DATA_DIR = 'testA/'

In [2]:
df = pd.read_hdf(DATA_DIR + 'total_exposure_math_fea.h5')
df.head()

ad_loc_id                                                            \
          count        mean        std    min    25%    50%    75%    max   
ad_id                                                                       
1          27.0  188.259259  52.691842   18.0  198.0  198.0  198.0  313.0   
2          75.0  146.560000  92.779343   39.0   56.0  123.0  196.0  297.0   
3          12.0  240.833333  47.668998  209.0  209.0  209.0  297.0  327.0   
4         602.0  208.162791  39.879854   30.0  199.0  209.0  209.0  323.0   
5           1.0   94.000000        NaN   94.0   94.0   94.0   94.0   94.0   

         bid                ...      total_ecpm               uid  \
       count        mean    ...             75%        max  count   
ad_id                       ...                                     
1       27.0  306.629630    ...      1027.25450   5311.008   27.0   
2       75.0  461.933333    ...      3365.49000  80089.520   75.0   
3       12.0   23.166667    ...       425.49675   7855.584   12.0   
4      602.0  173.282392    ...      1684.31250  13872.000  602.0   
5        1.0   12.000000    ...      8572.76000   8572.760    1.0   

                                                                    \
                mean            std       min        25%       50%   
ad_id                                                                
1      615381.777778  476662.410536    4784.0  119111.50  637025.0   
2      700073.173333  422250.156044     253.0  391392.00  695204.0   
3      644729.416667  426314.485286    6518.0  394688.75  666705.0   
4      686511.850498  398081.904489    3110.0  352644.75  685953.5   
5      260267.000000            NaN  260267.0  260267.00  260267.0   

                              
              75%        max  
ad_id                         
1       965421.50  1390924.0  
2       979732.00  1371564.0  
3       833585.75  1363790.0  
4      1022921.25  1395382.0  
5       260267.00   260267.0  

[5 rows x 72 columns]

In [3]:
df.columns

MultiIndex(levels=[[u'ad_loc_id', u'bid', u'expo_size', u'pctr', u'quality_ecpm', u'req_id', u'req_ts', u'total_ecpm', u'uid'], [u'count', u'mean', u'std', u'min', u'25%', u'50%', u'75%', u'max']],
           labels=[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8], [0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7]])

In [6]:
def col_with_prefix(col, prefix, ignore_cols=['ad_id']):
    if col in ignore_cols:
        return col
    return prefix + col
req_ts_fea = df['req_ts'].reset_index()
req_ts_fea.columns = [col_with_prefix(col, 'req_ts_') for col in req_ts_fea.columns]
req_ts_fea.head()

,ad_id,req_ts_count,req_ts_mean,req_ts_std,req_ts_min,req_ts_25%,req_ts_50%,req_ts_75%,req_ts_max
0,1,27.0,1.550727e+09,57732.494992,1.550637e+09,1.550664e+09,1.550763e+09,1.550770e+09,1.550787e+09
1,2,75.0,1.551238e+09,513733.210610,1.550472e+09,1.550736e+09,1.551468e+09,1.551707e+09,1.551791e+09
2,3,12.0,1.551966e+09,115836.447362,1.551875e+09,1.551926e+09,1.551928e+09,1.551930e+09,1.552264e+09
3,4,602.0,1.551486e+09,7176.895251,1.551436e+09,1.551485e+09,1.551486e+09,1.551488e+09,1.551495e+09
4,5,1.0,1.551093e+09,NaN,1.551093e+09,1.551093e+09,1.551093e+09,1.551093e+09,1.551093e+09


In [ ]:
# 
def make_libsvm(values, col, idx, id_set, id_len, single_col_set):
    if col in id_set:
        # 还要去重
        sorted_int_values = sorted(set((int(v) + idx for v in values.strip(',').split(',')
                                       if int(v) <= id_len
                                       )))
        return ' '.join(('%s:1' % v for v in sorted_int_values))
    if col in single_col_set:
        return '%s:%s' % (idx, values)
    raise Exception('unknow col: %s' % col)

def gen_svm_line(fea_cols, tup, label, single_col_set, idx_start_dict, fea_id_max):
    line = str(label)
    for idx, col in enumerate(fea_cols):
        values = str(tup[idx])
        idx_plus = idx_start_dict[col]
        line += ' ' + make_libsvm(values, col, idx_plus, id_set, id_len=fea_id_max.get(col, 0), single_col_set)
    return line